In [11]:
from cobra import *
from libsbml import *
import sys, getopt, os.path
from tqdm import tqdm
import zipfile
import tempfile

In [12]:
# create temporary directory
tempfile.tempdir = "./"
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir)

<TemporaryDirectory './tmp67vewp2a'>


In [6]:
omexFile = "SARS-CoV-2_CoreModelsBALF1Epithelial.omex"
sbmlPath = omexFile.replace(".omex","")
flattenedPath = omexFile.replace(".omex","_flattened")

In [7]:
# unzip OMEX file of interest in a temporary file
with zipfile.ZipFile(omexFile, 'r') as zip_ref:
    zip_ref.extractall(temp_dir.name+"/"+sbmlPath)

In [8]:
# create directory to store results
os.mkdir(flattenedPath)

In [9]:
# convert hierarchical models defined with the SBML Level 3 Hierarchical Model Composition package 
# into a "flattened" version

for model in os.listdir(temp_dir.name+"/"+sbmlPath):
    
    sbmldoc = readSBML(temp_dir.name+"/"+sbmlPath+"/"+model)
    
    # define converter options
    props = ConversionProperties()
    props.addOption("flatten comp", True)       

    # create "flattened" version
    result = sbmldoc.convert(props)
    if result != LIBSBML_OPERATION_SUCCESS:
        sbmldoc.printErrors()
        raise SystemExit("Conversion failed... %tb")
        
    writer  = SBMLWriter()
    writer.writeSBML(sbmldoc, flattenedPath+"/"+model.replace(".sbml",".xml"))
    
    print(model, "Flattening.... OK")


C149_C149_ACATCAGGTCGGGTCT-1_16782925330367706634.sbml Flattening.... OK
C148_C148_AGCTCCTCACGAAACG-1_1152717062397344256.sbml Flattening.... OK


In [10]:
# correct error: '' is not a valid SBML 'SId'.
for flatObj in os.listdir(flattenedPath):
    
    model = io.read_sbml_model(flattenedPath+"/"+flatObj)
    
    # Solve problem: '' is not a valid SBML 'SId'.
    model.id = str(flatObj).replace(".xml","").replace("-","_")
    
    print(model.id, "ID correction.... OK")
    
    # write corrected model (overwrite)
    io.write_sbml_model(model,flattenedPath+"/"+flatObj)

'' is not a valid SBML 'SId'.


C148_C148_AGCTCCTCACGAAACG_1_1152717062397344256 ID correction.... OK


'' is not a valid SBML 'SId'.


C149_C149_ACATCAGGTCGGGTCT_1_16782925330367706634 ID correction.... OK


In [13]:
# delete temporary file
temp_dir.cleanup()

### The final models can be used with any COBRApy or COBRAToolbox function.